In [1]:
import pathlib
import json
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve, auc, average_precision_score
from sklearn.utils import shuffle

import statsmodels.api as sm

In [2]:
# -------------------------
# Configuration / paths
# -------------------------
try:
    base_path = pathlib.Path(__file__).parent  # works when running as .py script
except NameError:
    base_path = pathlib.Path().resolve()       # works inside Jupyter or interactive mode

# The folder you said will contain all files:
# C:\Users\papad\OneDrive\Desktop\Study\MSc Data Science\25-26\HT25-1\Mathematical Modelling of Football\Final Test
# Put this script in that folder or run with working directory set to that folder.
players_path = base_path / 'players.json'
teams_path = base_path / 'teams.json'

# leagues and files
leagues = ["England", "France", "Germany", "Italy", "Spain"]
events_files = {league: base_path / f"events_{league}.json" for league in leagues}
matches_files = {league: base_path / f"matches_{league}.json" for league in leagues}

In [ ]:
# -------------------------
# Helper utilities (same as yours)
# -------------------------
def has_tag(tags, tag_id):
    return any(tag['id'] == tag_id for tag in tags)

def pos_to_meters(pos):
    if not isinstance(pos, list) or len(pos) == 0 or pos[0] is None:
        return np.nan, np.nan
    try:
        x_pct = pos[0].get('x', np.nan)
        y_pct = pos[0].get('y', np.nan)
    except Exception:
        return np.nan, np.nan
    x_m = x_pct * 105 / 100
    y_m = y_pct * 68 / 100
    return x_m, y_m

def add_xy(df):
    df = df.copy()
    df['x'] = df['positions'].apply(
        lambda p: p[0]['x'] * 105 / 100 if isinstance(p, list) and len(p) > 0 else np.nan
    )
    df['y'] = df['positions'].apply(
        lambda p: p[0]['y'] * 68 / 100 if isinstance(p, list) and len(p) > 0 else np.nan
    )
    return df

# Interaction helper (same as your function)
def add_interactions(df):
    df = df.copy()
    df['regain_x_centrality'] = df['regain_x'] * df['centrality']
    # comment lines omitted (kept minimal)
    return df

# -------------------------
# The core processing function — reproduces your full original pipeline for a single league.
# It returns several objects: cpev_df, player_CPEV (per-player aggregated), and the final_full_model/scaler (if training=True)
# -------------------------
def process_league(league, events_path, players_path, train_model=False, model_in=None, scaler_in=None, save_prefix=None):
    """
    Run the full pipeline for one league using your original code.
    If train_model=True -> trains and returns final_full_model and scaler.
    If train_model=False -> requires model_in and scaler_in to predict player CPEV.
    save_prefix if provided is used to name output CSVs.
    """
    print(f"\n--- Processing league: {league} ---")
    # Load events
    with open(events_path, encoding='utf-8') as f:
        data = json.load(f)

    event_df = pd.DataFrame(data)

    # Debug print (as in original)
    # print("Unique eventName values:", event_df['eventName'].unique())
    print("Unique subEventName values:", event_df['subEventName'].unique())

    # add x,y
    event_df = add_xy(event_df)

    # Turnovers
    turnover_conditions = (
        ((event_df['eventName'] == 'Pass') & event_df['tags'].apply(lambda x: has_tag(x, 1802))) |  # failed passes
        ((event_df['eventName'] == 'Duel') & event_df['tags'].apply(lambda x: has_tag(x, 1802))) |  # lost duels
        ((event_df['subEventName'] == 'Touch') & event_df['tags'].apply(lambda x: has_tag(x, 1302))) |  # missed touches
        (event_df['subEventName'] == 'Clearance')  # any clearance
    )
    df_turnovers = event_df[turnover_conditions].copy()
    df_turnovers['type'] = 'turnover'
    df_turnovers = add_xy(df_turnovers)

    # Regains
    regain_conditions = (
        (event_df['subEventName'].isin(['Ball recovery', 'Interception'])) |
        (event_df['eventName'] == 'Interruption') |  # sometimes interceptions recorded here
        ((event_df['eventName'] == 'Duel') & event_df['tags'].apply(lambda x: has_tag(x, 1801)))  # won duels
    )
    df_regains = event_df[regain_conditions].copy()
    df_regains['type'] = 'regain'
    df_regains = add_xy(df_regains)

    # Passes
    df_passes = event_df[event_df['eventName'] == 'Pass'].copy()
    df_passes['start_x'] = df_passes['positions'].apply(lambda pos: pos[0]['x'] if len(pos) > 0 else np.nan)
    df_passes['start_y'] = df_passes['positions'].apply(lambda pos: pos[0]['y'] if len(pos) > 0 else np.nan)
    df_passes['end_x'] = df_passes['positions'].apply(lambda pos: pos[-1]['x'] if len(pos) > 1 else np.nan)
    df_passes['end_y'] = df_passes['positions'].apply(lambda pos: pos[-1]['y'] if len(pos) > 1 else np.nan)
    df_passes['start_x_m'] = df_passes['start_x'] * 105 / 100
    df_passes['start_y_m'] = df_passes['start_y'] * 68 / 100
    df_passes['end_x_m']   = df_passes['end_x']   * 105 / 100
    df_passes['end_y_m']   = df_passes['end_y']   * 68 / 100
    df_passes['forward_distance'] = df_passes['end_x'] - df_passes['start_x']

    def is_progressive(row):
        if row['end_x_m'] <= row['start_x_m']:
            return False
        if row['start_x_m'] < 52.5 and row['end_x_m'] < 52.5:  # own half
            return row['forward_distance'] >= 30
        elif row['start_x_m'] < 52.5 and row['end_x_m'] >= 52.5:  # cross half
            return row['forward_distance'] >= 15
        elif row['start_x_m'] >= 52.5 and row['end_x_m'] >= 52.5:  # opponent half
            return row['forward_distance'] >= 10
        return False

    df_passes['is_progressive'] = df_passes.apply(is_progressive, axis=1)
    df_passes['is_successful'] = ~df_passes['tags'].apply(lambda x: has_tag(x, 1802))
    df_passes['is_successful_progressive'] = df_passes['is_progressive'] & df_passes['is_successful']
    df_passes['pass_distance'] = np.sqrt((df_passes['end_x'] - df_passes['start_x'])**2 + 
                                         (df_passes['end_y'] - df_passes['start_y'])**2)
    df_passes = df_passes[['matchId', 'teamId', 'playerId', 'subEventName', 'eventSec', 'matchPeriod',
                           "eventId", "id",
                           'start_x', 'start_y', 'end_x', 'end_y',
                           'pass_distance', 'forward_distance',
                           'is_progressive', 'is_successful', 'is_successful_progressive']]

    # Accelerations + reconstructed carries
    df_accels = event_df[event_df['subEventName'] == 'Acceleration'].copy()
    df_accels['start_x'] = df_accels['positions'].apply(lambda p: p[0]['x'] if len(p) > 0 else np.nan)
    df_accels['end_x']   = df_accels['positions'].apply(lambda p: p[-1]['x'] if len(p) > 1 else np.nan)
    df_accels['start_x_m'] = df_accels['start_x'] * 105 / 100
    df_accels['end_x_m']   = df_accels['end_x']   * 105 / 100
    df_accels['carry_distance_m'] = df_accels['end_x_m'] - df_accels['start_x_m']
    df_accels['is_progressive'] = df_accels['carry_distance_m'] >= 10
    df_accels['carry_type'] = 'acceleration'

    # reconstructed carries
    event_df = event_df.sort_values(['playerId', 'matchId', 'eventSec'])
    event_df['next_x'] = event_df.groupby('playerId')['x'].shift(-1)
    event_df['next_y'] = event_df.groupby('playerId')['y'].shift(-1)
    event_df['next_eventSec'] = event_df.groupby('playerId')['eventSec'].shift(-1)
    event_df['next_teamId'] = event_df.groupby('playerId')['teamId'].shift(-1)

    event_df['carry_distance'] = np.sqrt(
        (event_df['next_x'] - event_df['x'])**2 + (event_df['next_y'] - event_df['y'])**2
    )

    df_reconstructed = event_df[
        (event_df['next_teamId'] == event_df['teamId']) &
        ((event_df['next_eventSec'] - event_df['eventSec']) < 5)
    ].copy()

    df_reconstructed['start_x_m'] = df_reconstructed['x'] * 105 / 100
    df_reconstructed['end_x_m'] = df_reconstructed['next_x'] * 105 / 100
    df_reconstructed['carry_distance_m'] = df_reconstructed['end_x_m'] - df_reconstructed['start_x_m']
    df_reconstructed['is_progressive'] = df_reconstructed['carry_distance_m'] >= 10
    df_reconstructed['carry_type'] = 'Reconstructed Carry'

    df_carries = pd.concat([df_accels, df_reconstructed], ignore_index=True)

    df_carries = df_carries[[
        'id', 'matchId', 'teamId', 'playerId', 'eventSec',
        'start_x_m', 'end_x_m', 'carry_distance_m',
        'is_progressive', 'carry_type'
    ]]

    # Shots
    df_shots = event_df[event_df['eventName'] == 'Shot'].copy()
    df_shots = add_xy(df_shots)
    df_shots['in_penalty_area'] = (
        (df_shots['x'] >= 88.5) & (df_shots['x'] <= 105) &
        (df_shots['y'] >= 13.84) & (df_shots['y'] <= 54.16)
    )

    # Prepare indexing
    event_df.set_index('id', inplace=True, drop=False)
    df_passes.set_index('id', inplace=True, drop=False)
    df_carries.set_index('id', inplace=True, drop=False)

    cpev_rows = []

    # Main loop (matches)
    for match_id, match_turnovers in tqdm(df_turnovers.groupby('matchId'), desc="Processing matches", unit="match"):

        match_regains = df_regains[df_regains['matchId'] == match_id]
        match_events = event_df[event_df['matchId'] == match_id].sort_values('eventSec')
        match_shots = df_shots[df_shots['matchId'] == match_id]

        for _, turn in match_turnovers.iterrows():
            candidate_regains = match_regains[
                (match_regains['matchPeriod'] == turn['matchPeriod']) &
                (match_regains['teamId'] == turn['teamId']) &
                (match_regains['eventSec'] > turn['eventSec']) &
                (match_regains['eventSec'] - turn['eventSec'] <= 7)
            ].sort_values('eventSec')

            if candidate_regains.empty:
                continue

            regain = candidate_regains.iloc[0]

            next_team_events = match_events[
                (match_events['teamId'] == regain['teamId']) &
                (match_events['eventSec'] > regain['eventSec']) &
                (match_events['matchPeriod'] == regain['matchPeriod'])
            ].sort_values('eventSec')

            if not next_team_events.empty:
                next_event = next_team_events.iloc[0]
                next_action_type = next_event['eventName']
                next_subtype = next_event['subEventName']

                is_progressive = False
                pass_row = df_passes[df_passes['id'] == next_event['id']]
                carry_row = df_carries[df_carries['id'] == next_event['id']]

                if not pass_row.empty:
                    is_progressive = pass_row.iloc[0]['is_progressive']
                elif not carry_row.empty:
                    is_progressive = carry_row.iloc[0]['is_progressive']
            else:
                next_event = None
                next_action_type = None
                next_subtype = None
                is_progressive = False

            shots_window = match_shots[
                (match_shots['teamId'] == regain['teamId']) &
                (match_shots['matchPeriod'] == regain['matchPeriod']) &
                (match_shots['eventSec'] > regain['eventSec']) &
                (match_shots['eventSec'] - regain['eventSec'] <= 10)
            ]
            shot_in_box = not shots_window.empty and shots_window['in_penalty_area'].any()

            cpev_rows.append({
                'matchId': match_id,
                'turnover_teamId': turn['teamId'],
                'regain_teamId': regain['teamId'],
                'turnover_eventSec': turn['eventSec'],
                'regain_eventSec': regain['eventSec'],
                'time_to_regain': regain['eventSec'] - turn['eventSec'],
                'regain_x': regain['x'],
                'regain_y': regain['y'],
                'centrality': 1 - abs(regain['y'] - 34) / 34,
                'regain_playerId': regain['playerId'],
                'next_action_type': next_action_type,
                'next_action_subtype': next_subtype,
                'is_progressive_next_action': is_progressive,
                'shot_in_box_10s': shot_in_box
            })

    cpev_df = pd.DataFrame(cpev_rows)

    # Convert booleans
    cpev_df['is_progressive_next_action'] = cpev_df['is_progressive_next_action'].astype(int)
    cpev_df['shot_in_box_10s'] = cpev_df['shot_in_box_10s'].astype(int)

    # Time factor
    decay_rate = 0.2
    t = cpev_df['time_to_regain'].to_numpy()
    cpev_df['time_factor'] = np.exp(-decay_rate * t)

    # Sparsity (vectorized loop)
    relevant_keywords = [
        'Tackle', 'Interception', 'Clearance',
        'Ground defending duel', 'Air duel',
        'Simple pass', 'High pass', 'Smart pass', 'Ground attacking duel', 'Acceleration'
    ]

    events_def = event_df[event_df['subEventName'].isin(relevant_keywords)][
        ['matchId','teamId','eventSec','x','y']
    ].copy()

    radius_m = 25
    time_window = 10

    cpt_array = cpev_df[['matchId','regain_teamId','regain_eventSec','regain_x','regain_y']].to_numpy()
    sparsity_list = []

    for match_id, team_id, regain_time, regain_x, regain_y in tqdm(cpt_array, desc="Calculating sparsity"):
        mask = (
            (events_def['matchId'] == match_id) &
            (events_def['teamId'] != team_id) &
            (events_def['eventSec'] >= regain_time - time_window) &
            (events_def['eventSec'] < regain_time)
        )
        opp_events = events_def.loc[mask]

        if opp_events.empty:
            sparsity_list.append(1.0)
            continue

        dx = opp_events['x'].to_numpy() - regain_x
        dy = opp_events['y'].to_numpy() - regain_y
        dist = np.sqrt(dx**2 + dy**2)
        weights = np.exp(-dist / radius_m)

        sparsity_list.append(1 / (1 + weights.sum()))

    cpev_df['sparsity'] = sparsity_list

    # OSI
    cpev_df['OSI'] = cpev_df['time_factor'] * cpev_df['sparsity']
    cpev_df['OSI'] = (cpev_df['OSI'] - cpev_df['OSI'].min()) / (cpev_df['OSI'].max() - cpev_df['OSI'].min())

    print(cpev_df[['time_to_regain','time_factor','sparsity','OSI']].head())

    # -----------------------------------------
    # Features & target
    feature_cols = ['regain_x', 'centrality', 'OSI', 'is_progressive_next_action']
    target_col = 'shot_in_box_10s'

    X = cpev_df[feature_cols].copy()
    y = cpev_df[target_col].copy()

    # --- Train only on England ---
    if league.lower() == "england":
        print("\n⚽ Training model on England dataset...")
        train_model = True
    else:
        print("\n🌍 Evaluating model on", league, "midfielders using pretrained model.")
        train_model = False
    # add_interactions already defined above

    
    # Evaluate Logit with CV (re-using your original function logic)
    def evaluate_logit_auc_local(X_local, y_local, label="True", n_splits=5):
        kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
        roc_scores, pr_scores = [], []

        for fold, (train_idx, test_idx) in enumerate(kf.split(X_local, y_local), start=1):
            X_train, X_test = X_local.iloc[train_idx].copy(), X_local.iloc[test_idx].copy()
            y_train, y_test = y_local.iloc[train_idx], y_local.iloc[test_idx]

            scaler_local = MinMaxScaler()
            X_train['regain_x'] = scaler_local.fit_transform(X_train[['regain_x']])
            X_test['regain_x'] = scaler_local.transform(X_test[['regain_x']])

            X_train = add_interactions(X_train)
            X_test = add_interactions(X_test)

            X_train_sm = sm.add_constant(X_train, has_constant='add')
            X_test_sm = sm.add_constant(X_test, has_constant='add')

            model = sm.Logit(y_train, X_train_sm).fit(disp=False)
            y_pred = model.predict(X_test_sm)

            roc_auc = roc_auc_score(y_test, y_pred)
            pr_auc = average_precision_score(y_test, y_pred)

            roc_scores.append(roc_auc)
            pr_scores.append(pr_auc)

            print(f"✅ {label} | Fold {fold}: ROC-AUC={roc_auc:.3f}, PR-AUC={pr_auc:.3f}")

        return roc_scores, pr_scores


    # ---------------------------------------------------------------------
    # TRAINING SECTION — runs only for England
    # ---------------------------------------------------------------------
    if train_model:
        # Evaluate with CV
        roc_true, pr_true = evaluate_logit_auc_local(X, y, label="True")
        mean_roc_true, std_roc_true = np.mean(roc_true), np.std(roc_true)
        mean_pr_true, std_pr_true = np.mean(pr_true), np.std(pr_true)

        # Shuffle baseline
        y_shuffled = y.sample(frac=1, random_state=42).reset_index(drop=True)
        X_shuffled = X.reset_index(drop=True)
        roc_shuf, pr_shuf = evaluate_logit_auc_local(X_shuffled, y_shuffled, label="Shuffled")

        # Final model training
        X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)
        scaler_local_full = MinMaxScaler()
        X_train['regain_x'] = scaler_local_full.fit_transform(X_train[['regain_x']])
        X_test['regain_x'] = scaler_local_full.transform(X_test[['regain_x']])
        X_train = add_interactions(X_train)
        X_test = add_interactions(X_test)

        X_train_sm = sm.add_constant(X_train, has_constant='add')
        X_test_sm = sm.add_constant(X_test, has_constant='add')

        final_model = sm.Logit(y_train, X_train_sm).fit(disp=False)
        auc_test = roc_auc_score(y_test, final_model.predict(X_test_sm))

        print(f"\n✅ England model trained successfully | Test AUC = {auc_test:.3f}")

        # Fit final model on full England dataset
        scaler_full = MinMaxScaler()
        X_scaled = X.copy()
        X_scaled['regain_x'] = scaler_full.fit_transform(X_scaled[['regain_x']])
        X_scaled = add_interactions(X_scaled)
        X_scaled_sm = sm.add_constant(X_scaled, has_constant='add')

        final_full_model = sm.Logit(y, X_scaled_sm).fit(disp=False)
        print(final_full_model.summary())

    # ---------------------------------------------------------------------
    # EVALUATION SECTION — runs for other leagues using England model
    # ---------------------------------------------------------------------
    else:
        if (model_in is None) or (scaler_in is None):
            raise ValueError("For non-England leagues, please pass model_in and scaler_in.")

        final_full_model = model_in
        scaler_full = scaler_in

        print("\n✅ Using pretrained model from England for evaluation.")

    # -----------------------------
    # Predict CPEV per player using trained model (Rerun-Safe Version)
    # -----------------------------
    min_minutes = 900
    min_regains = 50

    cpev_df_model = cpev_df.copy()

    # Drop missing player IDs
    cpev_df_model = cpev_df_model.dropna(subset=['regain_playerId'])

    cpev_df_model['regain_x_scaled'] = scaler_full.transform(cpev_df_model[['regain_x']])
    cpev_df_model['regain_x_centrality'] = (
        cpev_df_model['regain_x_scaled'] * cpev_df_model['centrality']
    )

    feature_cols = [
        'regain_x_scaled',
        'centrality',
        'OSI',
        'is_progressive_next_action',
        'regain_x_centrality'
    ]

    X = cpev_df_model[feature_cols].copy()
    X_sm = sm.add_constant(X, has_constant='add')

    cpev_df_model['predicted_CPEV_prob'] = final_full_model.predict(X_sm)

    player_CPEV = (
        cpev_df_model.groupby('regain_playerId')['predicted_CPEV_prob']
        .sum()
        .reset_index()
        .rename(columns={'regain_playerId': 'playerId', 'predicted_CPEV_prob': 'CPEV'})
    )

    num_regains = (
        cpev_df.groupby('regain_playerId')
        .size()
        .reset_index(name='num_regains')
        .rename(columns={'regain_playerId': 'playerId'})
    )
    player_CPEV = player_CPEV.merge(num_regains, on='playerId', how='left')

    def adjust_event_seconds(row):
        return row['eventSec'] + 2700 if row['matchPeriod'] == '2H' else row['eventSec']

    event_df = event_df.copy()
    event_df['adjusted_eventSec'] = event_df.apply(adjust_event_seconds, axis=1)
    event_df['eventMin'] = event_df['adjusted_eventSec'] / 60

    player_match_times = event_df.groupby(['matchId', 'playerId']).agg(
        start_min=('eventMin', 'min'),
        end_min=('eventMin', 'max')
    ).reset_index()

    player_match_times['minutes_played'] = (
        player_match_times['end_min'] - player_match_times['start_min']
    )

    player_minutes = (
        player_match_times.groupby('playerId')['minutes_played']
        .sum()
        .reset_index()
        .rename(columns={'minutes_played': 'total_minutes'})
    )

    player_CPEV = player_CPEV.merge(player_minutes, on='playerId', how='left')

    player_CPEV['CPEV_per90'] = player_CPEV['CPEV'] / player_CPEV['total_minutes'] * 90

    # Load players
    with open(players_path, 'r', encoding='utf-8') as f:
        players_data = json.load(f)

    df_players_local = pd.json_normalize(players_data)
    df_players_local.rename(columns={'wyId': 'playerId'}, inplace=True)
    df_players_local['role'] = df_players_local['role.name']
    df_players_local['playerName'] = (
        df_players_local['firstName'].fillna('') + ' ' + df_players_local['lastName'].fillna('')
    ).str.replace('  ', ' ').str.strip()

    df_players_local = df_players_local[['playerId', 'playerName', 'role']]

    # Save pre-filter copy for all players
    player_CPEV_all = player_CPEV.copy()

    # Filter midfielders
    midfielders = df_players_local[df_players_local['role'] == 'Midfielder']
    player_CPEV = player_CPEV.merge(
        midfielders[['playerId', 'playerName']],
        on='playerId',
        how='inner'
    )

    player_CPEV = player_CPEV[
        (player_CPEV['total_minutes'] >= min_minutes) &
        (player_CPEV['num_regains'] >= min_regains)
    ]

    player_CPEV = player_CPEV.sort_values('CPEV_per90', ascending=False)

    print(player_CPEV[['playerName', 'total_minutes', 'num_regains', 'CPEV', 'CPEV_per90']].head(20))

    # Top 10 overall
    player_CPEV_all = player_CPEV_all.drop(columns=[col for col in ['playerName', 'role'] if col in player_CPEV_all.columns])
    player_CPEV_all = player_CPEV_all.merge(
        df_players_local[['playerId', 'playerName', 'role']],
        on='playerId',
        how='left'
    )
    player_CPEV_all = player_CPEV_all.dropna(subset=['playerName'])
    player_CPEV_all = player_CPEV_all[
        (player_CPEV_all['total_minutes'] >= min_minutes) &
        (player_CPEV_all['num_regains'] >= min_regains)
    ]
    top10_overall = player_CPEV_all.sort_values('CPEV_per90', ascending=False).head(10)

    print("\nTop 10 Players Overall (All Positions):")
    print(top10_overall[['playerName', 'role', 'total_minutes', 'num_regains', 'CPEV', 'CPEV_per90']].to_string(index=False))

    # Save outputs
    #out_mid = base_path / f"CPEV_midfielders_{league}.csv"
    #player_CPEV.to_csv(out_mid, index=False)
    #out_all = base_path / f"CPEV_{league}.csv"
    #player_CPEV_all.to_csv(out_all, index=False)
    #print(f"Saved midfielders to {out_mid}")
    #print(f"Saved all players to {out_all}")

    # If requested to return training artifacts
    if train_model:
        # Return cpev_df, trained model, scaler
        return cpev_df, final_full_model, scaler_full

    # otherwise return cpev and per-player DF
    return cpev_df, player_CPEV, player_CPEV_all

In [4]:
# -------------------------
# TRAIN on England (exactly using the same code path)
# -------------------------
print("\n=== TRAINING ON ENGLAND ===")
eng_events = events_files["England"]
cpev_eng, trained_model, trained_scaler = process_league(
    "England",
    eng_events,
    players_path,
    train_model=True
)

# Save model & scaler (same names you used)
#with open(base_path / "logit_CPEV_model.pkl", "wb") as f:
    #pickle.dump(trained_model, f)
#with open(base_path / "scaler.pkl", "wb") as f:
    #pickle.dump(trained_scaler, f)

#print("\nModel trained on England and saved as logit_CPEV_model.pkl and scaler.pkl")



=== TRAINING ON ENGLAND ===

--- Processing league: England ---
Unique subEventName values: ['Simple pass' 'High pass' 'Head pass' 'Air duel' 'Ground loose ball duel'
 'Smart pass' 'Launch' 'Ground defending duel' 'Ground attacking duel'
 'Foul' 'Free Kick' 'Cross' 'Shot' 'Reflexes' 'Touch' 'Clearance'
 'Ball out of the field' 'Throw in' 'Goal kick' 'Corner'
 'Goalkeeper leaving line' 'Hand pass' 'Acceleration' 'Save attempt' ''
 'Free kick cross' 'Free kick shot' 'Hand foul' 'Violent Foul' 'Protest'
 'Whistle' 'Late card foul' 'Out of game foul' 'Penalty' 'Time lost foul'
 'Simulation']


Calculating sparsity: 100%|██████████| 36913/36913 [02:31<00:00, 243.26it/s]


   time_to_regain  time_factor  sparsity       OSI
0        2.216387     0.641929  0.820088  0.504335
1        5.386256     0.340530  1.000000  0.308242
2        2.071649     0.660783  0.474807  0.279989
3        3.155139     0.532045  1.000000  0.510249
4        4.917617     0.373991  0.789516  0.260504

⚽ Training model on England dataset...
✅ True | Fold 1: ROC-AUC=0.886, PR-AUC=0.124
✅ True | Fold 2: ROC-AUC=0.874, PR-AUC=0.106
✅ True | Fold 3: ROC-AUC=0.874, PR-AUC=0.114
✅ True | Fold 4: ROC-AUC=0.883, PR-AUC=0.115
✅ True | Fold 5: ROC-AUC=0.873, PR-AUC=0.101
✅ Shuffled | Fold 1: ROC-AUC=0.495, PR-AUC=0.015
✅ Shuffled | Fold 2: ROC-AUC=0.474, PR-AUC=0.015
✅ Shuffled | Fold 3: ROC-AUC=0.473, PR-AUC=0.017
✅ Shuffled | Fold 4: ROC-AUC=0.523, PR-AUC=0.017
✅ Shuffled | Fold 5: ROC-AUC=0.497, PR-AUC=0.015

✅ England model trained successfully | Test AUC = 0.878
                           Logit Regression Results                           
Dep. Variable:        shot_in_box_10s   No. Obse

In [5]:
# -------------------------
# APPLY to other leagues (use the exact same pipeline; model not re-trained)
# -------------------------
for league in ["France", "Germany", "Italy", "Spain"]:
    cpev_df_league, player_CPEV_league, player_CPEV_all_league = process_league(
        league,
        events_files[league],
        players_path,
        train_model=False,
        model_in=trained_model,
        scaler_in=trained_scaler
    )

print("\n=== PIPELINE COMPLETE ===")


--- Processing league: France ---
Unique subEventName values: ['Simple pass' 'High pass' 'Air duel' 'Ground attacking duel'
 'Ground defending duel' 'Ground loose ball duel' 'Touch' 'Clearance'
 'Hand pass' 'Ball out of the field' 'Throw in' 'Head pass' 'Foul'
 'Free Kick' '' 'Free kick cross' 'Smart pass' 'Cross' 'Goal kick'
 'Acceleration' 'Shot' 'Reflexes' 'Launch' 'Corner'
 'Goalkeeper leaving line' 'Save attempt' 'Hand foul' 'Free kick shot'
 'Protest' 'Penalty' 'Out of game foul' 'Whistle' 'Simulation'
 'Violent Foul' 'Late card foul' 'Time lost foul']


Calculating sparsity: 100%|██████████| 34650/34650 [02:01<00:00, 284.80it/s]


   time_to_regain  time_factor  sparsity       OSI
0        3.055645     0.542738  0.386360  0.169012
1        1.346031     0.763986  1.000000  0.751958
2        1.705248     0.711024  0.867729  0.597348
3        2.427522     0.615387  0.708171  0.406807
4        2.011444     0.668788  0.645756  0.402678

🌍 Evaluating model on France using pretrained model.

✅ Using pretrained model from England for evaluation.
                            playerName  total_minutes  num_regains      CPEV  \
108                      Morgan Sanson    2120.089271           93  2.569423   
68                         Nabil Fekir    2487.654011          101  2.457449   
164    Andr\u00e9-Frank Zambo Anguissa    2424.148744          142  2.249847   
87                      Andrei Girotto    1512.777545           67  1.374728   
86     Wergiton do Ros\u00e1rio Calmon    1980.042619           97  1.772286   
148   Tanguy Ndomb\u00e8l\u00e9 Alvaro    2554.830719          107  2.282618   
69                      R

Calculating sparsity: 100%|██████████| 29880/29880 [01:02<00:00, 474.86it/s]


   time_to_regain  time_factor  sparsity       OSI
0        2.848580     0.565686  0.690437  0.361977
1        4.396206     0.415098  0.886916  0.338469
2        3.229522     0.524188  0.886916  0.439945
3        2.800534     0.571148  0.610616  0.318117
4        3.513782     0.495218  0.937897  0.439475

🌍 Evaluating model on Germany using pretrained model.

✅ Using pretrained model from England for evaluation.
                         playerName  total_minutes  num_regains      CPEV  \
143                     Suat Serdar    1667.887009          100  3.102865   
25               Lewis Harry Holtby    1176.494336           57  1.494175   
87                    Robin Quaison    1369.920810           54  1.659564   
40            Julian Baumgartlinger    1375.869909           65  1.592750   
142            Naby Deco Ke\u00efta    1908.914454          107  2.102336   
21                 Mario G\u00f6tze    1579.321775           81  1.730049   
67                      Danny Latza    2364.9

Calculating sparsity: 100%|██████████| 34140/34140 [02:06<00:00, 270.72it/s]


   time_to_regain  time_factor  sparsity       OSI
0        5.296485     0.346699  0.622959  0.187190
1        2.380646     0.621183  0.446305  0.251015
2        4.886121     0.376354  0.976832  0.345201
3        4.312772     0.422083  0.681416  0.261826
4        3.186094     0.528761  1.000000  0.513080

🌍 Evaluating model on Italy using pretrained model.

✅ Using pretrained model from England for evaluation.
                                    playerName  total_minutes  num_regains  \
92               Felipe Anderson Pereira Gomes    1170.776111           54   
36   Gast\u00f3n Exequiel Ram\u00edrez Pereyra    2183.194846          103   
91                      Allan Marques Loureiro    2845.296061          150   
134          Sergej Milinkovi\u0107-Savi\u0107    2888.021411          119   
18                             Bryan Cristante    2832.127345          112   
165                            Federico Chiesa    2999.433922          125   
95                      Artur Ioni\u0163

Calculating sparsity: 100%|██████████| 34510/34510 [01:53<00:00, 305.11it/s]


   time_to_regain  time_factor  sparsity       OSI
0        5.719028     0.318604  0.964133  0.272306
1        6.405860     0.277712  0.467114  0.084814
2        1.307085     0.769960  0.799151  0.597873
3        6.458465     0.274805  0.836215  0.190548
4        3.795285     0.468108  0.836215  0.361334

🌍 Evaluating model on Spain using pretrained model.

✅ Using pretrained model from England for evaluation.
                                     playerName  total_minutes  num_regains  \
128                           Gonzalo Escalante    1530.727965           97   
159                     Iv\u00e1n Alejo Peralta    1044.553711           59   
166                 Mauro Wilney Arambarri Rosa    2046.908655          106   
176                      Mikel Oyarzabal Ugarte    2714.718227           86   
140                     Joan Jord\u00e1n Moreno    2435.120185          104   
106                        Gabriel Appelt Pires    2368.968603          128   
102  Jos\u00e9 Paulo Bezzera Maci